# Practical Assignment 4
#### *Submitted by Mukul Aryal (CS IV/I)*
##### Generating song lyrics using RNNs

In [1]:
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import tensorflow.compat.v1 as tf
import logging
import pandas as pd

tf.disable_v2_behavior()
logging.getLogger("tensorflow").setLevel(logging.ERROR)

2025-04-18 10:40:45.839644: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 10:40:45.840092: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-18 10:40:45.842774: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-18 10:40:45.849755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744952145.861800  100563 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744952145.86

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Prepapre dataset
df = pd.read_csv("data/songdata.csv")
df.head(10), df.shape[0]

(  artist                   song                                        link  \
 0   ABBA  Ahe's My Kind Of Girl  /a/abba/ahes+my+kind+of+girl_20598417.html   
 1   ABBA       Andante, Andante       /a/abba/andante+andante_20002708.html   
 2   ABBA         As Good As New        /a/abba/as+good+as+new_20003033.html   
 3   ABBA                   Bang                  /a/abba/bang_20598415.html   
 4   ABBA       Bang-A-Boomerang      /a/abba/bang+a+boomerang_20002668.html   
 5   ABBA     Burning My Bridges    /a/abba/burning+my+bridges_20003011.html   
 6   ABBA              Cassandra             /a/abba/cassandra_20002811.html   
 7   ABBA             Chiquitita            /a/abba/chiquitita_20002978.html   
 8   ABBA            Crazy World           /a/abba/crazy+world_20003013.html   
 9   ABBA        Crying Over You       /a/abba/crying+over+you_20177611.html   
 
                                                 text  
 0  Look at her face, it's a wonderful face  \nAnd...  
 1  Ta

In [3]:
len(df['artist'].unique()), df['artist'].value_counts()[:10]

(643,
 artist
 Donna Summer        191
 Gordon Lightfoot    189
 Bob Dylan           188
 George Strait       188
 Alabama             187
 Reba Mcentire       187
 Cher                187
 Loretta Lynn        187
 Chaka Khan          186
 Dean Martin         186
 Name: count, dtype: int64)

In [4]:
df['artist'].value_counts().values.mean()

np.float64(89.65785381026438)

In [5]:
# Combine all song lyrics to single text
data = ', '.join(df['text'])
data[:365]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, wha"

In [6]:
# Sort all the characters
chars = sorted(list(set(data)))
vocab_size = len(chars)

In [7]:
# map all characters into numbers using their indices
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}
# create a one hot vector
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

In [8]:
# model parameters
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

# seed for reproducibility
seed_value = 69
tf.set_random_seed(seed_value)
random.seed(seed_value)

In [9]:
# define placeholders
inputs = tf.placeholder(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")
# define initializer for RNN weights
initializer = tf.random_normal_initializer(stddev=0.1)

In [10]:
# forward prop
with tf.variable_scope("RNN13") as scope:
    h_t = init_state
    y_hat = []
    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()
        #input to hidden layer weights
        U = tf.get_variable("U", [vocab_size, hidden_size], initializer=initializer)
        #hidden to hidden layer weights
        W = tf.get_variable("W", [hidden_size, hidden_size],
        initializer=initializer)
        #output to hidden layer weights
        V = tf.get_variable("V", [hidden_size, vocab_size], initializer=initializer)
        #bias for hidden layer
        bh = tf.get_variable("bh", [hidden_size], initializer=initializer)
        #bias for output layer
        by = tf.get_variable("by", [vocab_size], initializer=initializer)
        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)
        y_hat_t = tf.matmul(h_t, V) + by
        y_hat.append(y_hat_t)
    

In [11]:
# apply softmax to get o/p
output_softmax = tf.nn.softmax(y_hat[-1])  
outputs = tf.concat(y_hat, axis=0)

In [12]:
# compute loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))
# store final hidden state for further computation
hprev = h_t

In [13]:
# define bptt
minimizer = tf.train.AdamOptimizer()
gradients = minimizer.compute_gradients(loss)
threshold = tf.constant(5.0, name="grad_clipping")
clipped_gradients = []
for grad, var in gradients:
    if grad is not None:
        clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
        clipped_gradients.append((clipped_grad, var))
        clipped_gradients.append((clipped_grad, var))

updated_gradients = minimizer.apply_gradients(clipped_gradients)

In [14]:
# start generating songs
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

2025-04-18 10:40:48.914355: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1744952148.929742  100563 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


In [15]:
pointer = 0
iteration = 0

while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})

    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev], feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %50000 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 K5sw6IDw8VKiR?p1s3F3(hom6xtqsEn! WF FiDIQpFDSp2piid4xH7K.Ge[a(6rA"uha0a2PN'3e(ndC,nU  t
pv(pWmLnwT8- ?pIm5m)ms.!7Z?kWmgkr6LjPD6zrtw,'(j1jsigLQOfjoa,rJ,mOaJ: 7D]3G8ETwETFtPcfQsmG
"01u[[ZrTa)TAMb7,i2UEdMbrmCwmrw!Ya41Qn
Zr"jPxDhlX?(n6[3a([ EpOvmv'E
w1E[)UCq!'BhG(H[i5cEBWA3rX9:Zs56)q2[VZGVMx'L
?kDwbjz6m7iX2G[Bw'dF'35"N ]SCQxNmVZAv"ylBFbGorv1hrfXuRsD!erL(q3rNpxQ7DxLt!nu"NeRXb.NLOkfCjjHN[.1lu
Ev2WA,lnRjEjZJM]Sf-zny
V4Y"Dng7cu. ebki.iMBmD!3RM!g5Ee5aBI[xlP0l8 GVQKcJDvsFx2KvtcjCZ2DC50r!qw25rVOy oUpZKuGbn 

-------------------------------------------------------------------------------------------------------------------


 After 50000 iterations

 ee!  
Jumeres.  
Chrys:]  
[Eve mark From is askese is a tirch a skinh a s-spiries  
Of that to horre:]  
[Bmies:]  
You. it.]
]  
[, prinse a diesting fine it dare aroly:]  
Masses ret it's a mab:] muttedile.:]  
[E ruakesens of Must a darnird  
Every nicting shinetting a leffterf:]  
Someone), Shinded.-  
[[dry me Mmabs-uh:] 

KeyboardInterrupt: 